In [6]:
import sys

sys.path.append('/Users/valuamba/projs/components_agent_sales/app/utils')
sys.path.append('/Users/valuamba/projs/components_agent_sales/app/core/clients/')

from html_messages_parser import get_element_messages, get_messages_from_html_file, select_json_block 
from famaga import FamagaClient

In [95]:
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

In [28]:
from pathlib import Path
import hashlib
import subprocess
import json
from bs4 import BeautifulSoup


text_cache = Path('cache')

def sha1(input_string):
    """Helper to hash input strings"""
    try:

        # Step 5: Create a new SHA-1 hash object
        hash_object = hashlib.sha1()

        # Step 6: Update the hash object with the bytes-like object
        hash_object.update(input_string.encode('utf-8'))

        # Step 7: Get the hexadecimal representation of the hash
        return hash_object.hexdigest()
    except Exception as e:
        raise ValueError(input_string) from e

In [29]:
from functools import wraps
import inspect
import json
import pandas as pd
import hashlib


class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_dict'):
            return obj.to_dict()
        if isinstance(obj, pd.Int64Dtype):
            return int(obj)  # Convert Int64 to a regular int
        return json.JSONEncoder.default(self, obj)


def stored(func):
    """
    implements nix-like durable memoisation of function results.

    Lazy way to avoid recomputing expensive calls. Expects results to be JSON-serializable
    """
    @wraps(func)
    def CACHE(*args, **kwargs):
        name = func.__name__
        meta = {}

        meta["name"] = name
        meta["func"] = inspect.getsource(func)
        meta["args"] = args
        meta["kwargs"] = kwargs

        js = json.dumps(meta, cls=CustomEncoder)  # Using CustomEncoder
        sha = hashlib.sha1(js.encode('utf-8'))

        digest = sha.hexdigest()

        path = text_cache / f"{digest}-{name}.json"

        if path.exists():
            with path.open('r') as r:
                cached = json.load(r)
            return cached["result"]
        result = func(*args, **kwargs)
        meta["result"] = result
        with path.open('w') as w:
            json.dump(meta, w, cls=CustomEncoder)  # Using CustomEncoder
        return result

    return CACHE

In [24]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai_client = OpenAI()

@stored
def get_gpt(content, model="gpt-4-1106-preview", temperature=0, max_tokens=1000, stream=True):
    """
    Cached call to GPT.
    """
    messages = [{"role": "user", "content": content}]

    if stream:
        response = openai_client.chat.completions.create(
            model=model, 
            messages=messages, 
            temperature=temperature, 
            stream=True
        )
        
        collected_messages = []
        for chunk in response:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end='')
                collected_messages.append(chunk.choices[0].delta.content)
    
        content_str = ''.join(collected_messages)
        return content_str
    else:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.model_dump()


In [53]:
import requests
import logging
import pandas as pd
from typing import Dict, Any


BASE_URL = "https://api.famaga.org"


class APIClientV2:
    def __init__(self, token: str) -> None:
        self.token = token

    def _get_headers(self) -> Dict[str, str]:
        return {"Authorization": f"Bearer {self.token}"}

    def _handle_response(self, response: requests.Response) -> Dict:
        try:
            response.raise_for_status()
            print(f"Request to {response.url} succeeded with status code {response.status_code}")
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            raise
        except requests.exceptions.RequestException as e:
            print(f"Error occurred: {e}")
            raise

    @stored
    def _get(self, endpoint: str, params: Dict = None, limit: int = 100, page: int = 1) -> Dict:
        url = f"{BASE_URL}{endpoint}"
        if params is None:
            params = {}
        params['limit'] = limit
        params['page'] = page
        response = requests.get(url, headers=self._get_headers(), params=params)
        return self._handle_response(response)

    def _flatten_json(self, y: Dict[str, Any], parent_key: str = '', sep: str = '_') -> Dict[str, Any]:
        items = []
        for k, v in y.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(self._flatten_json(v, new_key, sep=sep).items())
            elif isinstance(v, list):
                for i, item in enumerate(v):
                    items.extend(self._flatten_json({f"{new_key}_{i}": item}).items())
            else:
                items.append((new_key, v))
        return dict(items)

    def offers_by_client_id(self, client_id: int, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get('/requisition', params={'client': client_id}, limit=limit, page=page)
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def offers_by_deal_id(self, deal_id: int, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get('/requisition', params={'request': deal_id}, limit=limit, page=page)
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def get_deals(self, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get('/requisition', params=None, limit=limit, page=page)
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def offer_products(self, offer_id: int, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get(f'/requisition/{offer_id}/products', limit=limit, page=page)
        flattened_data = [self._flatten_json(product) for product in data]
        return pd.DataFrame(flattened_data)

    def get_notice_mail(self, request_id: int, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get('/requisition-client-history', params={'request': request_id}, limit=limit, page=page)
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def _calculations_to_df(self, data):
        product_data = []
        
        for entry in data["content"]:
            if entry["version_data"]:
                calculation_id = entry["id"]
                request_id = entry["request"]["id"]
                created_at = entry["created_at"]

                for product in entry["version_data"]["products"]:
                    if isinstance(product, dict):
                        product_entry = {
                            "request_id": request_id,
                            "calculation_id": calculation_id,
                            "created_at": created_at,
                            **product
                        }
                        product_data.append(product_entry)

        product_df = pd.DataFrame(product_data)
        return product_df
        
    
    def get_calculation_history(self, request_id: int, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get('/calculation-history', params={'request': request_id}, limit=limit, page=page)
        return self._calculations_to_df(data)

    def get_calculations(self, limit: int = 100, page: int = 1) -> pd.DataFrame:
        data = self._get('/calculation-history', params=None, limit=limit, page=page)
        return self._calculations_to_df(data)

    def to_dict(self):
        return {
            "token": self.token
        }

In [54]:
client = APIClientV2('YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw')


In [36]:
deals_df = client.get_deals(limit=10000, page=2)

Request to https://api.famaga.org/requisition?limit=10000&page=2 succeeded with status code 200


In [57]:
calculations_df = client.get_calculations(limit=5000, page=1)

Request to https://api.famaga.org/calculation-history?limit=5000&page=1 succeeded with status code 200


In [43]:
sorted(deals_df['decision'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]

In [ ]:
## import pandas as pd
import json


client_id = 111636
# Step 1: Get unique request_id from client_purchase_history_df
client_purchase_history_df = client.offers_by_client_id(client_id)
unique_request_ids = client_purchase_history_df['request_id'].unique()

# Step 2: Build common DataFrame from request_ids with offer_products
offer_products_list = []
for request_id in unique_request_ids:
    offer_products_df = client.offer_products(request_id)
    offer_products_df['request_id'] = request_id  # Add request_id to each row
    offer_products_list.append(offer_products_df)
common_offer_products_df = pd.concat(offer_products_list, ignore_index=True)

# Step 3: Build common DataFrame from request_ids with get_notice_mail
notice_mail_list = []
for request_id in unique_request_ids:
    notice_mail_df = client.get_notice_mail(int(request_id))
    notice_mail_df['request_id'] = request_id  # Add request_id to each row
    notice_mail_list.append(notice_mail_df)
common_notice_mail_df = pd.concat(notice_mail_list, ignore_index=True)

# Step 4: Build common DataFrame from request_ids with get_calculation_history
calculation_history_list = []
for request_id in unique_request_ids:
    print(request_id)
    calculation_history_df = client.get_calculation_history(int(request_id))
    calculation_history_df['request_id'] = request_id  # Add request_id to each row
    calculation_history_list.append(calculation_history_df)
common_calculation_history_df = pd.concat(calculation_history_list, ignore_index=True)

In [ ]:
df = pd.DataFrame(data)

# Ensure created_at is a datetime column
df['created_at'] = pd.to_datetime(df['created_at'])

# Sort the DataFrame by created_at (date) in descending order and then by calculation_id in descending order
df = df.sort_values(by=['created_at', 'calculation_id'], ascending=[False, False])

def df_to_string(df):
    result = ""
    for request_id, req_group in df.groupby('request_id'):
        result += f"Deal {request_id}\n\n"
        for calculation_id, calc_group in req_group.groupby('calculation_id'):
            result += f"Calculation: {calc_group.iloc[0]['id']}\n"
            for _, row in calc_group.iterrows():
                result += f"{row['articul']} - {row['price_sell_ru']}\n"
            result += "\n"
    return result

result_string = df_to_string(df)
print(result_string)

In [45]:
class DecisionType:
    BILLED = 0
    BILL_PAID = 1
    PRICE = 2
    DELIVERY_TIME = 3
    REJECT_OTHER_REASON = 4
    TENDER = 5
    COLLECTING_OTHER_KP = 6
    THINK_OTHER_REASON = 7
    CONTRACT_SIGNING = 8
    OTHER_AGREEMENT = 9

YES_DECISIONS = [DecisionType.BILLED, DecisionType.BILL_PAID]
NO_DECISIONS = [DecisionType.PRICE, DecisionType.DELIVERY_TIME, DecisionType.REJECT_OTHER_REASON]
THINK_DECISIONS = [DecisionType.TENDER, DecisionType.COLLECTING_OTHER_KP, DecisionType.THINK_OTHER_REASON]
COORDINATION_DECISIONS = [DecisionType.CONTRACT_SIGNING, DecisionType.OTHER_AGREEMENT]

In [50]:
BLOCKED_FIRMS = [88704]

In [51]:
deals_df = deals_df[~deals_df['request_firm_id'].isin(BLOCKED_FIRMS)]


deals_df[deals_df['decision'].isin(YES_DECISIONS)]

,id,request_id,request_firm_id,request_firm_title,request_firm_type,request_firm_conversion,request_firm_link,request_initials,request_brand_id,request_brand_title,...,created,lastsend,processing_time,decision,link,target_price,target_price_solved,state,proform_fam,request_curator
3,373770,498091,124668,PMT-SYSTEMS,2,,index.php?option=com_oscrm&view=firmform&id=12...,KP,6222,BORRIES Markier-Systeme,...,2024-04-15T12:34:46+00:00,2024-04-15T12:53:02+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
4,373769,499139,84605,AGROAD s. r. o.,2,,index.php?option=com_oscrm&view=firmform&id=84...,EG,7639,BLOKSMA Engineering,...,2024-04-15T12:33:38+00:00,2024-04-15T12:53:05+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
15,373758,498023,124657,UAB PORIX,2,,index.php?option=com_oscrm&view=firmform&id=12...,KP,257,SOUTHCO,...,2024-04-15T12:03:33+00:00,2024-04-15T12:37:33+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
20,373753,498775,118172,PPHU IMEX,1,,index.php?option=com_oscrm&view=firmform&id=11...,KP,9706,EMILE MAURIN,...,2024-04-15T11:50:56+00:00,2024-04-15T11:52:48+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
24,373749,499135,89791,PANTATEC INDUSTRIAL Trading LLC,2,,index.php?option=com_oscrm&view=firmform&id=89...,EG,11661,ZD-MOTOR,...,2024-04-15T11:42:40+00:00,2024-04-15T11:54:34+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9982,363789,483981,67242,ADNOC,2,,index.php?option=com_oscrm&view=firmform&id=67...,EG,3652,AI-TEK Instruments,...,2024-02-26T08:31:41+00:00,2024-02-26T08:37:05+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
9984,363787,484058,122150,AMG Marek Górny sp.j.,1,,index.php?option=com_oscrm&view=firmform&id=12...,AKA,6795,WISTRO,...,2024-02-26T08:29:24+00:00,2024-02-26T08:42:34+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
9986,363785,468267,119017,APEXX SRL,1,,index.php?option=com_oscrm&view=firmform&id=11...,AKA,15999,Tecflam,...,2024-02-26T08:24:31+00:00,2024-03-06T12:59:21+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN
9990,363781,484076,29450,SilkRoad24 GmbH,3,7.14,index.php?option=com_oscrm&view=firmform&id=29...,AKA,8188,Polylux,...,2024-02-26T08:16:12+00:00,2024-02-26T08:20:18+00:00,,0,index.php?option=com_oscrm&view=requisition&re...,0.0,False,True,NaN,NaN


In [63]:
calculations_df['price_end'] = pd.to_numeric(calculations_df['price_end'], errors='coerce')
calculations_df['price_sell_ru'] = pd.to_numeric(calculations_df['price_sell_ru'], errors='coerce')

In [66]:
import numpy as np


def get_final_calculation_price(calculations_df):
    df = calculations_df.copy()

    df['price_final'] = np.where((df['price_end'].isna()) | (df['price_end'] == 0) | (df['price_end'] == 0.00), 
        df['price_sell_ru'], df['price_end'])

    return df

In [68]:
calculations_df

,request_id,calculation_id,created_at,id,tax,unit,count,weight,articul,expense,...,price_sell,brand_title,checked_out,contractor_id,discount_client,checked_out_time,confirmed_arrival_to,confirmed_arrival_from,planned_arrival_germany_to,planned_arrival_germany_from
0,513767,3040067,2024-06-03T15:18:15+00:00,1289677,7,set,1,10,10J162,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,513767,3040066,2024-06-03T15:17:38+00:00,1289677,7,set,1,10,10J162,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,514169,3040064,2024-06-03T15:15:55+00:00,1289715,7,pcs,1,13,0821401133 RD 80/100,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514169,3040063,2024-06-03T15:15:39+00:00,1289715,7,pcs,1,13,0821401133 RD 80/100,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,514441,3040062,2024-06-03T15:13:43+00:00,1289713,7,pcs,1,1,4V21008BG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7577,512786,3035073,2024-05-30T09:12:48+00:00,1287768,7,pcs,4,0.5,K32S.C6E,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7578,512786,3035072,2024-05-30T09:12:07+00:00,1287768,7,pcs,4,0.5,K32S.C6E,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7579,512786,3035071,2024-05-30T09:12:02+00:00,1287768,7,pcs,4,0.5,K32S.C6E,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7580,501928,3035069,2024-05-30T09:11:20+00:00,1287775,7,pcs,6,1,PMX-24,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
calculations_df = get_final_calculation_price(calculations_df)

request_calcs = calculations_df.groupby('request_id').size()\
    .reset_index(name='counts').sort_values(by='counts', ascending=False)

unique_request_ids = calculations_df['request_id'].unique()


for request_id in unique_request_ids:
    # print(request_id)
    
    calc_deal_df = calculations_df[calculations_df['request_id'] == request_id]

    discounted_requests = get_disconted_requests(calc_deal_df)

    if len(discounted_requests) > 1:
        pass
        # print(request_id)


grouped_calcs[grouped_calcs['counts'] < 20]

In [91]:
calc_deal_df = calculations_df[calculations_df['request_id'] == grouped_calcs.iloc[0]['request_id']]

In [ ]:
calc_deal_df['created_at'] = pd.to_datetime(calc_deal_df['created_at'])

calc_deal_df = calc_deal_df.sort_values(by='created_at', ascending=False)

calc_deal_df

In [136]:
calc_deal_df.loc[4709, 'price_final'] = 1000

In [ ]:
calc_deal_df.sort_values(by='created_at', ascending=True)

In [132]:
calc_deal_df.sort_values(by='created_at', ascending=False).loc[5, 'price_final'] = 1000

In [190]:

def get_sha_calc_row(row):
    return sha1(f'{str(row['articul'])}_{str(row['request_id'])}')

def get_disconted_requests(request_calcs):

    calc_deal_df = get_final_calculation_price(request_calcs)
    
    calc_deal_df['created_at'] = pd.to_datetime(calc_deal_df['created_at'])
    calc_deal_df = calc_deal_df.sort_values(by='created_at', ascending=False)
    
    # print(calc_deal_df['articul'].unique())

    discounted_calculations = []
    
    for index, row in calc_deal_df.groupby('articul'):
        # print(len(row))

        if len(row) > 1:
            latest_offer = row.iloc[0]
            first_offer = row.iloc[-1]

            # print(f'Latest offer: {latest_offer[['request_id', 'calculation_id', 'articul']]}')
            # print(f'First offer: {latest_offer[['request_id', 'calculation_id', 'articul']]}')
    
            # print(f'{get_sha_calc_row(latest_offer)} <> {get_sha_calc_row(first_offer)}')
        
            if first_offer['calculation_id'] != latest_offer['calculation_id']\
                and first_offer['articul'] == latest_offer['articul']\
                and first_offer['count'] == latest_offer['count']\
                and first_offer['price_final'] > latest_offer['price_final']:

                assert latest_offer['created_at'] > first_offer['created_at']
                assert latest_offer['calculation_id'] > first_offer['calculation_id']
                
                discounted_calculations.append({
                    'request_id': latest_offer['request_id'],
                    'articul':  latest_offer['articul']
                })
                print(f'({latest_offer['articul']}) {latest_offer['calculation_id']}) {latest_offer['price_final']} - ({first_offer['articul']}) {first_offer['price_final']}')
    
    discounted_calcs_df = pd.DataFrame(discounted_calculations)
    return request_calcs


In [159]:
calculations_df.sort_values(by='created_at', ascending=False)

,request_id,calculation_id,created_at,id,tax,unit,count,weight,articul,expense,...,brand_title,checked_out,contractor_id,discount_client,checked_out_time,confirmed_arrival_to,confirmed_arrival_from,planned_arrival_germany_to,planned_arrival_germany_from,price_final
0,513767,3040067,2024-06-03T15:18:15+00:00,1289677,7,set,1,10,10J162,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,658.43
1,513767,3040066,2024-06-03T15:17:38+00:00,1289677,7,set,1,10,10J162,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,640.17
2,514169,3040064,2024-06-03T15:15:55+00:00,1289715,7,pcs,1,13,0821401133 RD 80/100,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,977.12
3,514169,3040063,2024-06-03T15:15:39+00:00,1289715,7,pcs,1,13,0821401133 RD 80/100,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,977.12
4,514441,3040062,2024-06-03T15:13:43+00:00,1289713,7,pcs,1,1,4V21008BG,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7577,512786,3035073,2024-05-30T09:12:48+00:00,1287768,7,pcs,4,0.5,K32S.C6E,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128.57
7578,512786,3035072,2024-05-30T09:12:07+00:00,1287768,7,pcs,4,0.5,K32S.C6E,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128.57
7579,512786,3035071,2024-05-30T09:12:02+00:00,1287768,7,pcs,4,0.5,K32S.C6E,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128.57
7580,501928,3035069,2024-05-30T09:11:20+00:00,1287775,7,pcs,6,1,PMX-24,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.12


In [140]:
discounted_calcs_df

,request_id,articul
0,513733,52104486


In [ ]:
calc_deal_df.groupby('articul').first().reset_index()

In [ ]:
calculations_df[~calculations_df['price_sell'].isna()]